In [1]:
import pandas as pd
import numpy as np

In [2]:
security_df = pd.read_csv("./datasets/security_incidents.csv")

In [3]:
security_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4337 entries, 0 to 4336
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Incident ID               4337 non-null   int64  
 1   Year                      4337 non-null   int64  
 2   Month                     4289 non-null   float64
 3   Day                       3957 non-null   float64
 4   Country Code              4304 non-null   object 
 5   Country                   4332 non-null   object 
 6   Region                    3970 non-null   object 
 7   District                  3601 non-null   object 
 8   City                      3349 non-null   object 
 9   UN                        4337 non-null   int64  
 10  INGO                      4337 non-null   int64  
 11  ICRC                      4328 non-null   float64
 12  NRCS and IFRC             4328 non-null   float64
 13  NNGO                      4328 non-null   float64
 14  Other   

In [11]:
simple_df = security_df.head(2) 
simple_df.to_csv('./datasets/simple.csv', index=False)
simple_df

,Incident ID,Year,Month,Day,Country Code,Country,Region,District,City,UN,...,Attack context,Location,Latitude,Longitude,Motive,Actor type,Actor name,Details,Verified,Source
0,1,1997,1.0,NaN,KH,Cambodia,Banteay Meanchey,NaN,NaN,0,...,Unknown,Unknown,14.070929,103.099916,Unknown,Unknown,Unknown,1 ICRC national staff killed while working in ...,Archived,Archived
1,2,1997,1.0,NaN,RW,Rwanda,Northern,Musanze,Ruhengeri,0,...,Raid,Office/compound,-1.499840,29.634970,Unknown,Unknown,Unknown,"3 INGO international (Spanish) staff killed, 1...",Archived,Archived
